In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH


import subprocess
import os
import pathlib
import configparser
import sqlite3 as sl
import cv2
from pydub import AudioSegment
import math
import pickle
import shutil
import time
import multiprocessing
from multiprocessing import Process,Queue
import itertools
from threading import Thread
import soundfile as sf
from deepface import DeepFace
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


start_time = time.time()    # To measure execution time in seconds


print("PLEASE EDIT configuration.txt BEFORE EXECUTION")
print(".wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself")



datasetPathVideo =  configParser.get('COMMON', 'datasetPathVideo')
datasetPathAudio =  configParser.get('extractAudio', 'datasetPathAudio')
p =  configParser.get('extractAudio', 'dbChunk')
ttwbdf =  int(configParser.get('extractAudio', 'time_to_wait_before_deleting_files'))
cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
model_weights_path =  configParser.get('train_s2fe', 'model_weights_path')

# TODO dataset.db in configuration too

print("Video dataset at " + datasetPathVideo )
print("Number of cpus to use for multiprocessing : ", cpus)


  # Connection to databases
con = sl.connect(datasetPathDatabase,check_same_thread=False)
print('------------------- ABOUT TO START --------------------')

REQUIRED_SAMPLE_RATE = 16000
FACE_EMBEDDING_SIZE = 2622
AUDIO_MAX_LEN = 246000
len_to_input = 16000
NO_OF_VIDEOS = int(configParser.get('train_s2fe', 'no_of_videos'))
validation_split = float(configParser.get('train_s2fe', 'validation_split')) 
BATCH_SIZE = int(configParser.get('train_s2fe', 'batch_size'))
save_freq = int(configParser.get('train_s2fe', 'save_freq'))
no_of_epochs = int(configParser.get('train_s2fe', 'no_of_epochs'))


def read_audio_file(file_path):
  with open(file_path, "rb") as f:
      audio_wave, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  return audio_wave

 
def extractAudio(row):
    absPathVideo = row[0][1]   # for this one video

    absPathAudio = y = absPathVideo.replace(datasetPathVideo,datasetPathAudio)  # for this one audio
    absPathAudio = os.path.splitext(absPathAudio)[0]
    absPathAudio_w = absPathAudio   # without the end
    absPathAudio = absPathAudio + "_audio.wav"  # full path to extracted audio from the video

    #Create Directory
    pathlib.Path(os.path.dirname(absPathAudio)).mkdir(parents=True, exist_ok=True) 

    # Extract audio monochannel and with 16khz and put it in absPathAudio
    command = "ffmpeg -nostats -loglevel 0 -y -i '" + absPathVideo + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio + "'"
    subprocess.call(command, shell=True)


    # Get original duration of video
    audio = AudioSegment.from_file(absPathVideo)
    audio_length_og = math.floor(audio.duration_seconds)
    #print(audio_length_og)
    


    # Will either truncate or loop the original video to reach audio_length (3,6,12 or 24)
    audio_length_list = [6,12,24]
    for audio_length in audio_length_list:
        path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
        path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

        if(audio_length_og > audio_length):
            # Truncate    

            command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
            subprocess.call(command, shell=True)


        else:
            # Loop then truncaate
            #print("lesa")
            twoDigitLenStr = f"{audio_length:02}"
            #print(twoDigitLenStr)
            command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
            subprocess.call(command, shell=True)
            command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
            subprocess.call(command, shell=True)



            # Will delete those files after a little bit
        ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
        audio_wave = read_audio_file(path_var_len_audio)
        
        tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
        tDelete.start()
    return audio_wave
        
           




            
# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass
        


from random import randint
from scipy.fft import fft, ifft

def get_video(offset):
    
    try:
        #print('offset:' + str(offset))
        #print('offset:' + str(offset))
        data = con.execute("""SELECT V.ID, V.VIDEO_PATH, F.FACE_PATH FROM VIDEO V 
                        INNER JOIN FACE F ON F.ID = V.ID
                        LIMIT 1 OFFSET """ + str(offset))


        
        #print(data.fetchall())
        dataGotten = data.fetchall()
        if (len(dataGotten)) == 0:
            raise ValueError('No video was fetched')

        #print(dataGotten[0][0])
        audio_wave = extractAudio(dataGotten)
        audio_wave = audio_wave[0:len_to_input]
        audio_wave = tf.constant(audio_wave, dtype=tf.float64)

        #audio = tf.squeeze(audio_wave, axis=-1)
        fft = tf.signal.fft(
            tf.cast(tf.complex(real=audio_wave, imag=tf.zeros_like(audio_wave)), tf.complex64)
        )
        #fft = tf.expand_dims(fft, axis=-1)
        #fft = tf.squeeze(fft, axis=1)
        #print()
        fft = fft[0:(audio_wave.shape[0] // 2)]
        fft = tf.math.abs(fft)
        #fft = tf.expand_dims(fft, axis=-1)

        embedding_objs = DeepFace.represent(dataGotten[0][2],enforce_detection=False)
        return fft,tf.constant(embedding_objs[0]['embedding'], dtype=tf.float64)
    except Exception as e:
        print('Error getting video, retrying ...:' + str(offset))
        return get_video(randint(1, NO_OF_VIDEOS - 1))


    








2024-03-12 21:58:25.601012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


PLEASE EDIT configuration.txt BEFORE EXECUTION
.wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself
Video dataset at /home/gamal/Datasets/Dataset1/Video
Number of cpus to use for multiprocessing :  8
------------------- ABOUT TO START --------------------


In [2]:
import numpy as np
#np.set_printoptions(threshold=1000000)
get_video(1)

(<tf.Tensor: shape=(8000,), dtype=float32, numpy=
 array([2.6001568 , 2.2688131 , 2.3923457 , ..., 0.33533525, 0.51442325,
        0.63863385], dtype=float32)>,
 <tf.Tensor: shape=(2622,), dtype=float64, numpy=
 array([ 0.01234863,  0.0142761 ,  0.00426896, ..., -0.00633709,
         0.01058111,  0.01097511])>)

In [3]:
import numpy as np

def valid_generator(n):

    
    n =  int(NO_OF_VIDEOS * (1-validation_split))
    # initialize counter
    value = -1

    
    result_array = np.arange(n, NO_OF_VIDEOS)
    np.random.shuffle(result_array)

    # loop until counter is less than n
    while value < len(result_array) - 1:
        #print(value)
        #print(len(result_array))
        value = value + 1
        yield get_video(result_array[value])


def data_generator(n):

    n =  int(NO_OF_VIDEOS * (1-validation_split))
    # initialize counter
    value = -1

    
    result_array = np.arange(1, n)
    np.random.shuffle(result_array)

    # loop until counter is less than n
    while value < len(result_array) - 1:
        #print(value)
        #print(n)
        value = value + 1
        yield get_video(result_array[value])

        # increment the counter
        



for x in valid_generator(20):
    print(x)

In [4]:
import os

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)


TF version: 2.10.0


In [5]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import shutil
import numpy as np

import tensorflow as tf
import keras

from pathlib import Path
from IPython.display import display, Audio

SAMPLING_RATE = 16000

In [6]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_outputs):
    inputs = keras.layers.Input(shape=input_shape, name="input")
    inputs2 = keras.layers.BatchNormalization()(inputs)
    inputs3 = keras.layers.Reshape(target_shape=(input_shape,1))(inputs2)
    
    x = residual_block(inputs3, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)

    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(num_outputs, activation="linear", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((len_to_input//2), 2622)

model.summary()

# Compile the model using Adam's default learning rate


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000)]       0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 8000)        32000       ['input[0][0]']                  
 alization)                                                                                       
                                                                                                  
 reshape (Reshape)              (None, 8000, 1)      0           ['batch_normalization[0][0]']    
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['reshape[0][0]']          

In [7]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float64),
    tf.TensorSpec(shape=(None), dtype=tf.float64),
)

dataset = tf.data.Dataset.from_generator(data_generator, args=[NO_OF_VIDEOS], 
                                         output_signature=output_signature)

dataset_valid = tf.data.Dataset.from_generator(valid_generator, args=[NO_OF_VIDEOS], 
                                         output_signature=output_signature)




In [8]:
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

dataset_valid = dataset_valid.batch(BATCH_SIZE)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)



In [9]:
num_train_batches = int(NO_OF_VIDEOS / BATCH_SIZE)
train_dataset = dataset.take(num_train_batches)
valid_dataset = dataset_valid.take(num_train_batches)


In [10]:
try:
    model.load_weights(model_weights_path)
    print('Loaded weights')
except:
    print('Could not load weights, using random ones.')

Could not load weights, using random ones.


In [11]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= model_weights_path,
    save_weights_only=True,
    save_freq = save_freq,

    )


In [12]:
from gradient_accumulator import GradientAccumulateOptimizer
LEARNING_RATE = 0.0001
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
optimizer = GradientAccumulateOptimizer(accum_steps=5, optimizer=optimizer)

In [13]:
model.compile(optimizer, loss=loss_fn)

In [17]:
bb = get_video(5)
print(bb[1])
model.predict(tf.expand_dims(bb[0], axis=0))

tf.Tensor(
[ 0.01010858  0.0079855   0.00182851 ... -0.00792103  0.01460108
  0.0016924 ], shape=(2622,), dtype=float64)
1/1 [==============================] - 0s 105ms/step


array([[-0.02355498,  0.01846472, -0.05685584, ...,  0.08099563,
         0.09888003,  0.04346417]], dtype=float32)

In [18]:
bb = get_video(1)
print(bb[1])
model.predict(tf.expand_dims(bb[0], axis=0))

tf.Tensor(
[ 0.01234863  0.0142761   0.00426896 ... -0.00633709  0.01058111
  0.01097511], shape=(2622,), dtype=float64)
1/1 [==============================] - 0s 78ms/step


array([[-0.06879631, -0.15874718,  0.00033301, ...,  0.01943459,
        -0.03036234,  0.02665952]], dtype=float32)

In [19]:
history_model = model.fit(train_dataset,validation_data = valid_dataset
                          , epochs=no_of_epochs,callbacks=[model_checkpoint_callback])
history_model

Epoch 1/1000
2/2 [==============================] - 25s 11s/step - loss: 0.0036 - val_loss: 0.0051
Epoch 2/1000
2/2 [==============================] - 24s 11s/step - loss: 0.0041 - val_loss: 0.0052
Epoch 3/1000
2/2 [==============================] - 25s 12s/step - loss: 0.0030 - val_loss: 0.0052
Epoch 4/1000
2/2 [==============================] - 25s 12s/step - loss: 0.0067 - val_loss: 0.0052
Epoch 5/1000
2/2 [==============================] - 25s 11s/step - loss: 0.0035 - val_loss: 0.0052
Epoch 6/1000
2/2 [==============================] - 25s 11s/step - loss: 0.0021 - val_loss: 0.0051
Epoch 7/1000
2/2 [==============================] - 25s 11s/step - loss: 0.0024 - val_loss: 0.0051
Epoch 8/1000
2/2 [==============================] - 25s 11s/step - loss: 0.0020 - val_loss: 0.0051
Epoch 9/1000
2/2 [==============================] - 25s 12s/step - loss: 0.0016 - val_loss: 0.0050
Epoch 10/1000
2/2 [==============================] - 25s 12s/step - loss: 0.0049 - val_loss: 0.0050
Epoch 11/

In [ ]:
history_model